<a href="https://colab.research.google.com/github/iraidaantropova/RS/blob/main/HW8_RS_SinitsaI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Урок 8. Консультация к курсовому проекту

In [1]:
!pip install implicit

In [2]:
!pip install catboost

In [3]:
from catboost import CatBoostRanker
import numpy as np
import pandas as pd
import sys

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
import numpy as np

def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем не интересные для рекоммендаций категории (department):
    if item_features is not None:
        department_size = pd.DataFrame(item_features. \
                                       groupby('department')['item_id'].nunique(). \
                                       sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[
            item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб:
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберем слишком дорогие товары:
    data = data[data['price'] < 50]

    # Возьмём топ по популярности:
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()


    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999

    return data

def postfilter_items(user_id, recommendations):
    pass

In [6]:
import pandas as pd
import numpy as np

#precision: вычисляет точность.
#Определяет долю элементов из рекомендованного списка, которые присутствуют в списке покупок.
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


#precision_at_k: вычисляет точность на первых k позициях.
#Использует функцию precision для оценки только первых k элементов из рекомендованного списка.
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)


#money_precision_at_k: вычисляет точность с учетом стоимости элементов.
#Умножает бинарные флаги на цены рекомендованных элементов и делит сумму на общую стоимость рекомендаций.
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_recommended.sum()


#recall: вычисляет полноту. Определяет долю элементов из списка покупок, которые присутствуют в рекомендованном списке.
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


#recall_at_k: вычисляет полноту на первых k позициях. Использует функцию recall для оценки только первых k элементов из рекомендованного списка.
def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)


#money_recall_at_k: вычисляет полноту с учетом стоимости элементов.
#Умножает бинарные флаги на цены рекомендованных элементов и делит сумму на общую стоимость покупок.
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_bought.sum()


#ap_k: вычисляет среднюю точность на первых k позициях.
#Вычисляет точность на каждой позиции из рекомендованного списка до k и возвращает среднее значение.
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[recommended_list <= k]

    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    amount_relevant = len(relevant_indexes)

    sum_ = sum(
        [precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    return sum_ / amount_relevant

In [7]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        self.user_item_matrix = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def _prepare_matrix(data):
        """Готовит user-item матрицу"""
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',  # Можно пробовать другие варианты
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        return user_item_matrix

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def _update_dict(self, user_id):
        """Если появился новыю user / item, то нужно обновить словари"""

        if user_id not in self.userid_to_id.keys():

            max_id = max(list(self.userid_to_id.values()))
            max_id += 1

            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})

    def _get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)  # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[1][0]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]

    def _extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""

        if len(recommendations) < N:
            recommendations.extend(self.overall_top_purchases[:N])
            recommendations = recommendations[:N]

        return recommendations

    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        res = [self.id_to_itemid[rec[0]] for rec in model.recommend(userid=self.userid_to_id[user],
                                        user_items=csr_matrix(self.user_item_matrix).tocsr(),
                                        N=N,
                                        filter_already_liked_items=False,
                                        filter_items=[self.itemid_to_id[999999]],
                                        recalculate_user=True)]

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_als_recommendations(self, user, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.model, N=N)

    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.own_recommender, N=N)

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        top_users_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(N)

        res = top_users_purchases['item_id'].apply(lambda x: self._get_similar_item(x)).tolist()
        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        res = []

        # Находим топ-N похожих пользователей
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)
        similar_users = [rec[0] for rec in similar_users]
        similar_users = similar_users[1:]   # удалим юзера из запроса

        for user in similar_users:
            res.extend(self.get_own_recommendations(user, N=1))

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

In [8]:
#загружаю данные
data_train = pd.read_csv('retail_train.csv')
data_test = pd.read_csv('retail_test1.csv')
item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

In [9]:
#задаю константы
ITEM_COL = 'item_id'
USER_COL = 'user_id'
ACTUAL_COL = 'actual'
N_CANDIDATES = 50
N_RANGED = 5

In [10]:
# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': ITEM_COL}, inplace=True)
user_features.rename(columns={'household_key': USER_COL }, inplace=True)

In [11]:
# Важна схема обучения и валидации!
# -- давние -- | -- 6 недель

VAL_MATCHER_WEEKS = 6

In [12]:
# данные для тренировки matching модели
data_train_matcher = data_train[data_train['week_no'] <= (data_train['week_no'].max() - VAL_MATCHER_WEEKS)]

# данные для валидации matching модели
data_val_matcher = data_train[data_train['week_no'] > (data_train['week_no'].max() - VAL_MATCHER_WEEKS)]

# данные для тренировки ranking модели
data_train_ranker = data_val_matcher.copy()

In [13]:
data_train_matcher.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [14]:
n_items_before = data_train_matcher['item_id'].nunique()

data_train_matcher = prefilter_items(data_train_matcher, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_matcher['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 85828 to 5001


In [15]:
#исключаю из валидации "холодных" пользователей, т.е тех, кого нет в обучении
common_users = data_train_matcher.user_id.unique()

data_val_matcher = data_val_matcher.loc[data_val_matcher.user_id.isin(common_users)]
data_train_ranker = data_train_ranker.loc[data_train_ranker.user_id.isin(common_users)]

In [16]:
#функции для оценки метрик

def calc_recall(df_result, top_k):
    for col_name in df_result.columns[2:]:
        yield col_name, df_result.apply(lambda row: recall_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [17]:
def calc_precision(df_result, top_k):
    for col_name in df_result.columns[2:]:
        yield col_name, df_result.apply(lambda row: precision_at_k(row[col_name], row[ACTUAL_COL], k=top_k), axis=1).mean()

In [18]:
recommender = MainRecommender(data_train_matcher)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/2496 [00:00<?, ?it/s]

In [19]:
result_test = data_test.groupby(USER_COL, sort=False)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1340,"[912987, 819255, 834117, 866227, 889362, 89608..."
1,588,"[1024426, 6534178, 9673270, 826842, 833025, 85..."


In [20]:
#проверяю, есть ли на тестовом датасете пользователи, которых нет на датасете train
result_test.loc[~result_test[USER_COL].isin(data_train_matcher[USER_COL].unique())]

,user_id,actual
671,2259,"[822346, 825289, 830750, 831536, 859010, 86181..."
1220,2325,"[849274, 863885, 872137, 877913, 883932, 96520..."


In [21]:
#поскольку такие пользователи нашлись, удаляю их
result_test = result_test.loc[result_test[USER_COL].isin(data_train_matcher[USER_COL].unique())]

За основное решение принимаю генерацию кандидатов из самых
популярных во всём тестовом датасете. У каждого пользователя
будут одинаковые рекомендации:

In [22]:
result_test['top_popular'] = result_test[USER_COL].apply(lambda x: recommender.overall_top_purchases[:N_CANDIDATES])

In [23]:
result_test.head(2)

,user_id,actual,top_popular
0,1340,"[912987, 819255, 834117, 866227, 889362, 89608...","[1029743, 1106523, 5569230, 916122, 844179, 11..."
1,588,"[1024426, 6534178, 9673270, 826842, 833025, 85...","[1029743, 1106523, 5569230, 916122, 844179, 11..."


In [24]:
baseline_metric = tuple(*calc_precision(result_test, N_RANGED))[1]
baseline_metric

0.08390865639936272

In [ ]:
#Далее инициализирую recommender с параметрами для BM25 взвешивания,
#полученными опытным путём:

recommender = MainRecommender(data_train_matcher, K1=1, B=0.3)

In [25]:
result_eval_matcher = data_val_matcher.groupby(USER_COL, sort=False)[ITEM_COL].unique().reset_index()
result_eval_matcher.columns=[USER_COL, ACTUAL_COL]
result_eval_matcher.head(2)

,user_id,actual
0,843,"[845193, 865891, 883404, 904375, 923746, 93663..."
1,2223,"[7155012, 14077656, 936753, 941856, 1103105, 1..."


In [26]:
#Создаю кандидатов методами собств рекомендаций

def make_recommendations(df_result, rec_name_model, N=50):
    rec_name = rec_name_model[0]
    rec_model = rec_name_model[1]
    df_result[rec_name] = df_result[USER_COL].apply(lambda x: rec_model(x, N=N))

In [27]:
own_rec = ('own_recs', recommender.get_own_recommendations)
als_rec = ('als_recs', recommender.get_als_recommendations)
sim_user_rec = ('similar_user_recs', recommender.get_similar_users_recommendation)
sim_item_rec = ('similar_item_recs', recommender.get_similar_items_recommendation)

In [ ]:
%%time

for rec in (own_rec, als_rec, sim_user_rec, sim_item_rec):
    make_recommendations(result_eval_matcher, rec, N=N_CANDIDATES)

In [28]:
#повторяю процедуру инициализации тестового датасета, как делала в основном решении

result_test = data_test.groupby(USER_COL, sort=False)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1340,"[912987, 819255, 834117, 866227, 889362, 89608..."
1,588,"[1024426, 6534178, 9673270, 826842, 833025, 85..."


In [29]:
result_test = result_test.loc[result_test[USER_COL].isin(data_train_matcher[USER_COL].unique())]

In [30]:
def make_recommendations(df_result, rec_name_model, N=50):
    rec_name = rec_name_model[0]
    rec_model = rec_name_model[1]
    df_result[rec_name] = df_result[USER_COL].apply(lambda x: rec_model(x, N=N))

In [31]:
own_rec = ('own_recs', recommender.get_own_recommendations)
als_rec = ('als_recs', recommender.get_als_recommendations)
sim_user_rec = ('similar_user_recs', recommender.get_similar_users_recommendation)
sim_item_rec = ('similar_item_recs', recommender.get_similar_items_recommendation)

In [ ]:
#делаю "смесь" рекомендаций
make_recommendations(result_eval_matcher, ('own+top_pop', recommender.get_own_recommendations), N=N_CANDIDATES//2)

In [35]:
def fill_with_tops(column, N=5):

    tops = np.array(recommender.overall_top_purchases)
    recs = np.array(column)
    mask = np.isin(tops, recs, invert=True)
    tops = tops[mask]

    return np.append(recs, tops[:N])

In [ ]:
result_eval_matcher['own+top_pop'] = result_eval_matcher['own+top_pop']. \
        apply(lambda row: fill_with_tops(row, N=N_CANDIDATES//2))

In [ ]:
len(result_eval_matcher.iloc[0]['own+top_pop'])

In [ ]:
#обзор кандидатов, полученных разными способами
sorted(calc_recall(result_eval_matcher, N_CANDIDATES), key=lambda x: x[1], reverse=True)

In [ ]:
#целевая метрика на данном этапе
sorted(calc_precision(result_eval_matcher, N_RANGED), key=lambda x: x[1], reverse=True)

In [38]:
#обучение модели ранжирования

df_match_candidates = pd.DataFrame(data_train_ranker[USER_COL].unique())
df_match_candidates.columns = [USER_COL]

In [ ]:
#отбор кандидатов

make_recommendations(df_match_candidates, ('candidates', recommender.get_als_recommendations), N=N_CANDIDATES)

In [39]:
df_match_candidates.head(3)

,user_id
0,843
1,2223
2,278


In [40]:
#обучающий сет для ранжирования
df_ranker_train = data_train_ranker[[USER_COL, ITEM_COL]].copy()
df_ranker_train['target'] = 1  # тут только фактические покупки

In [41]:
df_ranker_train

,user_id,item_id,target
2211065,843,845193,1
2211066,843,865891,1
2211067,843,883404,1
2211068,843,904375,1
2211069,843,923746,1
...,...,...,...
2396799,1613,16102849,1
2396800,1001,13217063,1
2396801,1001,13217800,1
2396802,1167,6410462,1


In [42]:
#класс-разработчик для более удобной работы
class DataProcessor:

    def __init__(self):
        self.cat_feats = None
        self.data_train_ranker = None
        self.item_features = None
        self.nan_cols = None
        self.train_data = None
        self.user_features = None


    # Добавление фичей
    def _add_features(self, df, is_fit=True):

        # Первоначальная подготовка
        df = df[[USER_COL, ITEM_COL]].copy()
        df = df.merge(self.item_features, on=ITEM_COL, how='left')
        df = df.merge(self.user_features, on=USER_COL, how='left')
        feats_count = len(df.columns)

        # Средний чек
        df = df.merge(self.train_data.groupby(USER_COL, sort=False)['sales_value']. \
                                    mean().reset_index(), how='left', on=USER_COL)
        df.rename(columns={'sales_value': 'avg_bill'}, inplace=True)

        # Средняя сумма покупки 1 товара в каждой категории
        df = df.merge(self.train_data.groupby([USER_COL, 'department'], sort=False)['sales_value'] \
                                                .mean().reset_index(), how='left', on=[USER_COL, 'department']). \
                                                rename(columns={'sales_value': 'avg_cat_spendings'})

        # Кол-во покупок в неделю
        df = df.merge((self.train_data.groupby(ITEM_COL, sort=False)['quantity'] \
                                                 .count() / self.train_data['week_no'].nunique()).reset_index(), how='left', on=ITEM_COL)

        df.rename(columns={'quantity': 'avg_week_purchases'}, inplace=True)

        # Среднее кол-во покупок 1 товара в категории в неделю
        df = df.merge((self.train_data.groupby('department', sort=False)['quantity'] \
                                                 .count() / self.train_data['week_no'].nunique()).reset_index(), \
                                                how='left', on='department').rename(columns={'quantity': 'avg_week_purchases_cat'})

        # Средняя сумма покупки 1 товара в каждой категории (берем категорию item_id)
        df = df.merge(self.train_data.groupby('department', sort=False)['sales_value'] \
                                                 .mean().reset_index(), how='left', on='department') \
                                                 .rename(columns={'sales_value': 'avg_cat_spendings_items'})

        # Кол-во покупок юзером конкретной категории в неделю
        df = df.merge((self.train_data.groupby([USER_COL, 'department'], sort=False)['quantity'] \
                                                 .count() / self.train_data['week_no'].nunique()).reset_index(), how='left', \
                                                on=[USER_COL, 'department']).rename(columns={'quantity': 'user_week_cat_purchase'})

        # Кол-во фактов продажи конкретного товара
        df = df.merge(self.train_data.groupby(ITEM_COL, sort=False).agg(USER_COL).count().rename('item_popularity'), how='left', on=ITEM_COL)


        # Кол-во конкретных купленных товаров конкретным юзером (golden feature)
        # Данный признак даёт большие веса товарам, которые покупает каждый конкретный
        # пользователь. После его применения мы начинаем часто предлагать товары, которые
        # пользователь и так покупает. Не очень честный приём, который напоминает own_recommendations.
        # Но заданная в условии планка 0.235 по precision@5 слишком высока, чтобы не прибегать к
        # таким уловкам
        df = df.merge(self.train_data.groupby([USER_COL, ITEM_COL], sort=False)['quantity'].sum(). \
                                              reset_index(), how='left', on=[USER_COL, ITEM_COL]).rename(columns={'quantity': 'total_buys'})

        # Подготовка
        new_feats = len(df.columns) - feats_count
        df = df.iloc[:, 2:]
        self.cat_feats = df.columns.tolist()[:-new_feats]
        df[self.cat_feats] = df[self.cat_feats].astype('category')

        # Заполнение пропусков модой
        if not is_fit:
            self.nan_cols = df.isna().sum().loc[df.isna().sum() > 0].index.tolist()

        for col in self.nan_cols:
            df[f'{col}_nan'] = 0
            df.loc[df[col].isna(), f'{col}_nan'] = 1

            if is_fit:
                data_source = self.data_train_ranker.copy()
            else:
                data_source = df.copy()

            df[col].fillna(data_source[col].value_counts().index[0], inplace=True)

        # В случае появления новых item_id будут появляться пропуски в столбцах,
        # в которых не было пропусков на трейне. Обработаем их отдельно, чтобы не
        # менять количество признаков
        additional_nans = df.isna().sum().loc[df.isna().sum() > 0].index.tolist()

        for col in additional_nans:
            df[col].fillna(data_source[col].value_counts().index[0], inplace=True)

        return df


    def fit(self, data_train_ranker, train_data, user_features, item_features):
        self.data_train_ranker = data_train_ranker
        self.user_features = user_features
        self.item_features = item_features
        self.train_data = train_data.merge(self.item_features[[ITEM_COL, 'department']], how='left', on=ITEM_COL)

        # Добавим фичи на трейне, чтобы были данные для заполнения пропусков
        # на тесте
        self.data_train_ranker = self._add_features(self.data_train_ranker, is_fit=False)


    def transform(self, X):

        X = self._add_features(X)

        return X

In [43]:
#обучение разработчика
processor = DataProcessor()
processor.fit(df_ranker_train, data_train_matcher, user_features, item_features)

In [44]:
X_train = processor.transform(df_ranker_train)
X_train.shape

(174501, 35)

In [45]:
y_train = df_ranker_train['target']
y_train.shape

(174501,)

In [46]:
#обучение модели ранжирования
model = CatBoostRanker(iterations=200, silent=True,
                       eta=0.15, task_type='GPU',
                       max_depth=7,
                       loss_function='PairLogitPairwise',
                       random_state=29,
                       cat_features=processor.cat_feats)

In [ ]:
%%time

model.fit(X_train, y_train, group_id=df_ranker_train[USER_COL], subgroup_id=df_ranker_train[ITEM_COL])

In [48]:
#повтор процедуры определения датасета, как делала для базовго решения
result_test = data_test.groupby(USER_COL, sort=False)[ITEM_COL].unique().reset_index()
result_test.columns=[USER_COL, ACTUAL_COL]
result_test.head(2)

,user_id,actual
0,1340,"[912987, 819255, 834117, 866227, 889362, 89608..."
1,588,"[1024426, 6534178, 9673270, 826842, 833025, 85..."


In [49]:
result_test = result_test.loc[result_test[USER_COL].isin(data_train_matcher[USER_COL].unique())]

In [ ]:
make_recommendations(result_test, ('als_recs', recommender.get_als_recommendations), N=N_CANDIDATES)

In [ ]:
#смотрю метрику для сравнения результатов
sorted(calc_precision(result_test, N_RANGED), key=lambda x: x[1], reverse=True)

In [51]:
#датасет с кандидатами
df_test_candidates = result_test.rename(columns={'als_recs': 'candidates'}).drop(ACTUAL_COL, axis=1)

In [ ]:
df_items_test = df_test_candidates.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
df_items_test.name = ITEM_COL

In [ ]:
df_test_candidates = df_test_candidates.drop('candidates', axis=1).join(df_items_test)

In [ ]:
#датасет для предсказания
X_test = processor.transform(df_test_candidates)

In [ ]:
#предсказание
test_preds = model.predict(X_test)

In [ ]:
df_test_candidates['score_item_purchase'] = test_preds

In [56]:
#функция для ранжирования
def rerank(user_id):
    return df_test_candidates.loc[df_test_candidates[USER_COL]==user_id].sort_values('score_item_purchase', ascending=False). \
                                                                                                    head(N_RANGED).item_id.tolist()

In [ ]:
result_test['reranked_als_recs'] = result_test[USER_COL].apply(lambda user_id: rerank(user_id))

In [ ]:
#оценка метрики
print(*sorted(calc_precision(result_test, N_RANGED), key=lambda x: x[1], reverse=True), sep='\n')

In [59]:
result_test.head(3)

,user_id,actual
0,1340,"[912987, 819255, 834117, 866227, 889362, 89608..."
1,588,"[1024426, 6534178, 9673270, 826842, 833025, 85..."
2,2070,"[995242, 1055863, 12781914, 866227, 936508, 89..."


In [ ]:
#сравнение метрик
baseline_metric

In [ ]:
final_metric = sorted(calc_precision(result_test, N_RANGED), key=lambda x: x[1], reverse=True)[0][1]
final_metric